# PageIndex + seekdb 混合 RAG 系统 - 完整演示

本笔记本演示如何使用 PageIndex + seekdb 混合 RAG 系统进行文档索引和检索。

## 📋 演示内容

1. **环境设置** - 导入库和配置
2. **系统初始化** - 启动 seekdb 和初始化组件
3. **文档索引** - 使用 PageIndex 解析并索引 PDF
4. **检索演示** - 三种检索策略对比
5. **结果可视化** - 可视化检索结果
6. **高级用法** - 权重调优和配置

---

## 1. 环境设置

导入必要的库和模块。

In [ ]:
# 系统库
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# 添加项目路径
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# 核心模块
from src.config import config
from src.document_indexer import DocumentIndexer
from src.hybrid_search import HybridSearchEngine, HybridSearchConfig, TreeSearchConfig, VectorSearchConfig
from src.seekdb_manager import SeekDBManager
from src.embedding_manager import EmbeddingManager
from src.cache_manager import CacheManager

# 可视化和数据处理
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown, HTML

# 日志
from loguru import logger
logger.remove()  # 移除默认处理器
logger.add(sys.stdout, level="INFO", format="<green>{time:HH:mm:ss}</green> | <level>{message}</level>")

# 设置绘图样式
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ 所有库导入成功")

## 2. 检查系统状态

检查 seekdb 和配置状态。

In [ ]:
import subprocess

def check_seekdb_status():
    """检查 seekdb Docker 容器状态"""
    try:
        result = subprocess.run(
            ['docker', 'ps', '--filter', 'name=seekdb', '--format', '{{.Status}}'],
            capture_output=True,
            text=True
        )
        if result.stdout.strip():
            return True, result.stdout.strip()
        return False, "容器未运行"
    except Exception as e:
        return False, str(e)

# 检查 seekdb
is_running, status = check_seekdb_status()
if is_running:
    display(Markdown(f"### ✅ seekdb 状态: 运行中\n```\n{status}\n```"))
else:
    display(Markdown(f"### ⚠️ seekdb 状态: {status}\n\n请运行: `docker-compose up -d`"))

# 显示配置信息
config_info = f"""
### 📋 系统配置

- **seekdb 模式**: {config.seekdb.seekdb_mode}
- **seekdb 地址**: {config.seekdb.seekdb_host}:{config.seekdb.seekdb_port}
- **数据库**: {config.seekdb.seekdb_database}
- **Embedding 模型**: {config.openai.openai_embedding_model}
- **向量维度**: {config.seekdb.embedding_dims}
- **缓存**: {'启用' if config.cache.enable_cache else '禁用'}
"""
display(Markdown(config_info))

## 3. 初始化系统组件

初始化 seekdb、embedding 和检索引擎。

In [ ]:
# 1. 初始化 seekdb 管理器
logger.info("初始化 seekdb 管理器...")
db_manager = SeekDBManager(
    mode=config.seekdb.seekdb_mode,
    persist_directory=config.seekdb.seekdb_persist_dir,
    host=config.seekdb.seekdb_host,
    port=config.seekdb.seekdb_port,
    user=config.seekdb.seekdb_user,
    password=config.seekdb.seekdb_password,
    database=config.seekdb.seekdb_database
)

# 2. 初始化 embedding 管理器
logger.info("初始化 Embedding 管理器...")
embed_manager = EmbeddingManager(
    api_key=config.openai.get_api_key(),
    model=config.openai.openai_embedding_model,
    base_url=config.openai.base_url
)

# 3. 初始化缓存管理器（可选）
cache_manager = None
if config.cache.enable_cache:
    logger.info("初始化缓存管理器...")
    cache_manager = CacheManager(
        seekdb_manager=db_manager,
        embedding_manager=embed_manager,
        enable_cache=True,
        ttl=config.cache.cache_ttl
    )

# 4. 初始化检索引擎
logger.info("初始化混合检索引擎...")
search_engine = HybridSearchEngine(
    seekdb_manager=db_manager,
    embedding_manager=embed_manager,
    cache_manager=cache_manager
)

# 5. 初始化文档索引器
logger.info("初始化文档索引器...")
document_indexer = DocumentIndexer(
    openai_api_key=config.openai.get_api_key(),
    seekdb_mode=config.seekdb.seekdb_mode,
    embedding_dims=config.seekdb.embedding_dims,
    persist_directory=config.seekdb.seekdb_persist_dir,
    seekdb_host=config.seekdb.seekdb_host,
    seekdb_port=config.seekdb.seekdb_port,
    seekdb_user=config.seekdb.seekdb_user,
    seekdb_password=config.seekdb.seekdb_password,
    seekdb_database=config.seekdb.seekdb_database,
    base_url=config.openai.base_url,
    model_name=config.openai.model_name
)

logger.success("✓ 所有组件初始化完成")

## 4. 文档索引演示

使用 PageIndex 解析 PDF 并索引到 seekdb。

**注意**: 这个过程可能需要几分钟，取决于文档大小。

In [ ]:
# 设置文档信息
pdf_path = project_root / "data" / "1282-1311_存储架构.pdf"
document_id = "demo_storage_architecture"

if not pdf_path.exists():
    display(Markdown(f"### ⚠️ PDF 文件不存在: {pdf_path}\n请确保文件存在后重新运行。"))
else:
    display(Markdown(f"### 📄 索引文档\n\n- **文件**: {pdf_path.name}\n- **大小**: {pdf_path.stat().st_size / 1024:.2f} KB\n- **文档ID**: {document_id}"))
    
    # 执行索引
    logger.info("开始索引文档...")
    result = document_indexer.index_document(
        pdf_path=str(pdf_path),
        document_id=document_id
    )
    
    # 显示结果
    display(Markdown(f"""
### ✅ 索引完成

| 指标 | 数量 |
|------|------|
| 总节点数 | {result['total_nodes']} |
| 总内容块 | {result['total_chunks']} |
| 文档页数 | {result['total_pages']} |
"""))

## 5. 检索演示 - 三种策略对比

对比三种检索策略的效果：
- **tree_only**: 仅使用树结构检索（基于文档层级）
- **vector_only**: 仅使用向量检索（语义相似度）
- **hybrid**: 混合检索（推荐，准确率最高）

In [ ]:
# 定义查询
query = "什么是LSM-Tree存储架构？它有什么特点？"

display(Markdown(f"### 🔍 查询: *{query}*"))

# 执行三种策略的检索
strategies = ["tree_only", "vector_only", "hybrid"]
all_results = {}

for strategy in strategies:
    logger.info(f"执行 {strategy} 检索...")
    results = search_engine.hybrid_search(
        query=query,
        document_id=document_id,
        strategy=strategy,
        top_k=5
    )
    all_results[strategy] = results
    logger.success(f"✓ {strategy}: 返回 {len(results)} 个结果")

print("\n所有检索完成！")

### 5.1 查看各策略的 Top-3 结果

In [ ]:
def display_results(strategy_name, results, top_k=3):
    """展示检索结果"""
    display(Markdown(f"\n#### 📊 {strategy_name.upper()} 策略 - Top {top_k} 结果\n"))
    
    for i, result in enumerate(results[:top_k], 1):
        path_str = " > ".join(result.node_path) if result.node_path else "N/A"
        
        result_md = f"""
##### 结果 {i} (分数: {result.score:.4f})

**章节路径**: {path_str}  
**页码**: {result.page_num}

**内容**:
```
{result.content[:300]}...
```

---
"""
        display(Markdown(result_md))

# 显示各策略的结果
for strategy in strategies:
    display_results(strategy, all_results[strategy], top_k=3)

### 5.2 结果对比分析

In [ ]:
# 创建对比数据
comparison_data = []
for strategy in strategies:
    results = all_results[strategy]
    for i, result in enumerate(results[:5], 1):
        comparison_data.append({
            '策略': strategy,
            '排名': i,
            '分数': result.score,
            '页码': result.page_num,
            '内容长度': len(result.content)
        })

df = pd.DataFrame(comparison_data)

# 显示表格
display(Markdown("### 📈 策略对比表"))
display(df)

# 统计信息
stats = df.groupby('策略').agg({
    '分数': ['mean', 'max', 'min'],
    '排名': 'count'
}).round(4)

display(Markdown("\n### 📊 统计摘要"))
display(stats)

## 6. 结果可视化

可视化各策略的检索效果。

In [ ]:
# 创建可视化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 图1: 各策略的平均分数对比
avg_scores = df.groupby('策略')['分数'].mean().sort_values(ascending=False)
avg_scores.plot(kind='bar', ax=axes[0], color=['#3498db', '#e74c3c', '#2ecc71'])
axes[0].set_title('各策略平均分数对比', fontsize=14, fontweight='bold')
axes[0].set_ylabel('平均分数', fontsize=12)
axes[0].set_xlabel('检索策略', fontsize=12)
axes[0].set_ylim([0, 1])
axes[0].grid(True, alpha=0.3)

# 在柱状图上显示数值
for i, v in enumerate(avg_scores):
    axes[0].text(i, v + 0.02, f'{v:.4f}', ha='center', fontweight='bold')

# 图2: Top-5 结果分数分布
for strategy in strategies:
    strategy_data = df[df['策略'] == strategy]
    axes[1].plot(strategy_data['排名'], strategy_data['分数'], 
                marker='o', label=strategy.upper(), linewidth=2, markersize=8)

axes[1].set_title('Top-5 结果分数趋势', fontsize=14, fontweight='bold')
axes[1].set_xlabel('排名', fontsize=12)
axes[1].set_ylabel('相关性分数', fontsize=12)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)
axes[1].set_xticks([1, 2, 3, 4, 5])

plt.tight_layout()
plt.show()

# 显示结论
best_strategy = avg_scores.idxmax()
best_score = avg_scores.max()

display(Markdown(f"""
### 🏆 检索效果分析

- **最佳策略**: {best_strategy.upper()} (平均分数: {best_score:.4f})
- **推荐**: 对于大多数查询，**HYBRID** 策略提供最佳的准确率和召回率平衡
- **Tree-only**: 适合结构化查询（如"第三章讲了什么"）
- **Vector-only**: 适合语义查询（如"如何优化性能"）
"""))

## 7. 高级用法 - 权重调优

演示如何通过调整权重来优化检索效果。

In [ ]:
# 测试不同的权重组合
weight_configs = [
    (0.7, 0.3, "重树结构"),
    (0.5, 0.5, "平衡"),
    (0.3, 0.7, "重向量")
]

weight_results = []

display(Markdown("### ⚙️ 测试不同权重配置\n"))

for tree_w, vector_w, desc in weight_configs:
    config_obj = HybridSearchConfig(
        tree_weight=tree_w,
        vector_weight=vector_w
    )
    
    results = search_engine.hybrid_search(
        query=query,
        document_id=document_id,
        strategy="hybrid",
        top_k=5,
        config=config_obj
    )
    
    avg_score = sum(r.score for r in results[:5]) / min(5, len(results))
    weight_results.append({
        '配置': desc,
        '树权重': tree_w,
        '向量权重': vector_w,
        '平均分数': avg_score,
        'Top-1分数': results[0].score if results else 0
    })

weight_df = pd.DataFrame(weight_results)
display(weight_df)

# 可视化权重影响
fig, ax = plt.subplots(figsize=(10, 5))
x = range(len(weight_df))
width = 0.35

ax.bar([i - width/2 for i in x], weight_df['平均分数'], width, label='平均分数', color='#3498db')
ax.bar([i + width/2 for i in x], weight_df['Top-1分数'], width, label='Top-1分数', color='#e74c3c')

ax.set_xlabel('权重配置', fontsize=12)
ax.set_ylabel('分数', fontsize=12)
ax.set_title('不同权重配置的检索效果', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(weight_df['配置'])
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 8. 多查询测试

测试系统对不同类型查询的响应。

In [ ]:
# 定义不同类型的查询
test_queries = [
    ("什么是LSM-Tree？", "概念性查询"),
    ("存储架构的主要组件有哪些？", "枚举性查询"),
    ("如何优化存储性能？", "方法性查询"),
    ("文档的第一章讲了什么？", "结构性查询")
]

display(Markdown("### 🧪 多查询测试\n"))

multi_query_results = []

for query_text, query_type in test_queries:
    display(Markdown(f"\n#### 查询: *{query_text}* ({query_type})\n"))
    
    # 使用 hybrid 策略
    results = search_engine.hybrid_search(
        query=query_text,
        document_id=document_id,
        strategy="hybrid",
        top_k=3
    )
    
    if results:
        top_result = results[0]
        multi_query_results.append({
            '查询': query_text,
            '类型': query_type,
            'Top-1分数': top_result.score,
            '页码': top_result.page_num,
            '结果数': len(results)
        })
        
        display(Markdown(f"""
**Top-1 结果** (分数: {top_result.score:.4f})  
页码: {top_result.page_num} | 路径: {' > '.join(top_result.node_path)}

```
{top_result.content[:200]}...
```
"""))

# 显示汇总
multi_df = pd.DataFrame(multi_query_results)
display(Markdown("\n### 📋 查询结果汇总"))
display(multi_df)

## 9. 系统统计

查看系统的总体统计信息。

In [ ]:
# 获取统计信息
stats = db_manager.get_statistics()

display(Markdown(f"""
### 📊 系统统计

| 指标 | 数值 |
|------|------|
| 总节点数 | {stats.get('total_nodes', 'N/A')} |
| 总内容块 | {stats.get('total_chunks', 'N/A')} |
| Collections | {', '.join(stats.get('collections', []))} |
"""))

# 列出所有文档
documents = db_manager.list_documents()
if documents:
    docs_df = pd.DataFrame(documents)
    display(Markdown("\n### 📚 已索引文档列表"))
    display(docs_df)
else:
    display(Markdown("\n### 📚 暂无已索引文档"))

## 10. 总结

本笔记本演示了 PageIndex + seekdb 混合 RAG 系统的核心功能：

### ✅ 完成的演示

1. **系统初始化** - seekdb、embedding、检索引擎
2. **文档索引** - PageIndex 解析 + seekdb 存储
3. **三种检索策略** - tree_only / vector_only / hybrid
4. **结果可视化** - 分数对比、趋势分析
5. **权重调优** - 自定义 tree/vector 权重
6. **多查询测试** - 不同类型查询的效果

### 🎯 关键发现

- **Hybrid 策略**通常提供最佳效果
- **权重调优**可以针对特定查询类型优化
- **Tree 检索**适合结构化查询
- **Vector 检索**适合语义查询

### 📖 进一步探索

- 尝试索引自己的 PDF 文档
- 调整检索配置参数
- 使用 API 服务进行集成
- 查看 `docs/api.md` 了解更多用法

---

**项目仓库**: https://github.com/boathell/pageindex-seekdb-rag  
**文档**: [README.md](../README.md) | [API 文档](../docs/api.md)